In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import utils as vutils
import numpy as np
import os
from pathlib import Path

# our implement
from model import Net
from myDataset import MyDataset

In [2]:

weights_root = Path("weights")
weights_root.mkdir(exist_ok=True)

transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    ])
data_list = ['normal', 'bad_leg']
mydataset = MyDataset(data_list, transforms_)
train_len = int(len(mydataset)*0.8)
test_len = len(mydataset) - train_len
train_set, val_set = torch.utils.data.random_split(mydataset, [train_len, test_len])

normal
bad_leg
(7666, 3, 25)


In [3]:
len(train_set)

6132

In [8]:
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device", torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()))




net = Net()
net.to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.002)
criterion  = nn.CrossEntropyLoss()


transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    ])

# prepare data    
data_list = ['normal', 'bad_leg']
mydataset = MyDataset(data_list, transforms_)
train_len = int(len(mydataset)*0.8)
test_len = len(mydataset) - train_len
train_set, test_set = torch.utils.data.random_split(mydataset, [train_len, test_len])


train_loader = DataLoader(
    train_set,
    batch_size=64,
    shuffle=True,
    pin_memory=True
)
test_loader = DataLoader(
    test_set,
    batch_size=64,
    shuffle=True,
    pin_memory=True
)


device 0 NVIDIA GeForce RTX 2080 with Max-Q Design
normal
bad_leg
(7666, 3, 25)


In [9]:
print(test_set[0][0][0].shape)

torch.Size([3, 25])


In [10]:
# test_net = Net()
# test_net.to(device)

# data = test_set[0][0].to(device)
# test_net.load_state_dict(torch.load(PATH))
# output = test_net(data)
# output

In [11]:
training_start = time.time()
for epoch in range(1,50):
    loss_all = 0
    net.train()
    for i, batch in enumerate(train_loader):
        
        
        data = batch[0].to(device)
        label = batch[1].to(device)
        optimizer.zero_grad()
        output = net(data).squeeze()
        loss = criterion(output, label.long())
        loss.backward()
        loss_all += loss.item()
        optimizer.step()
        
        print(f'\rloss: {loss.item()}',end='')
        # print('\r',loss_all/(i+epoch*len(dataloader)))
        

    acc = 0
    for i, batch in enumerate(test_loader):
        
        net.eval()
        data = batch[0].to(device)
        label = batch[1].to(device)
        
        output = net(data).squeeze()
        #output = output.topk(1)[1].squeeze()
        acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
        
    print(' acc: ',acc/(i+1))
    torch.save(net.state_dict(), weights_root / f"E{epoch+1:03}.pth")
        
    
        
        

       


loss: 0.67750275135040283 acc:  0.5226604502688171
loss: 0.34158545732498174 acc:  0.8676705309139785
loss: 0.21969176828861237 acc:  0.9120043682795699
loss: 0.20565393567085266 acc:  0.9289524529569891
loss: 0.078807085752487185 acc:  0.953061995967742
loss: 0.170291334390640264 acc:  0.9244161626344086
loss: 0.043043989688158035 acc:  0.9621975806451614
loss: 0.155208110809326175 acc:  0.9465935819892474
loss: 0.097201175987720494 acc:  0.9074260752688171
loss: 0.192376196384429935 acc:  0.939390120967742
loss: 0.150473102927207954 acc:  0.9732862903225806
loss: 0.160983070731163025 acc:  0.9543850806451614
loss: 0.089920639991760254 acc:  0.976541498655914
loss: 0.0907776802778244 acc:  0.9680779569892474
loss: 0.155678614974021965 acc:  0.9660828293010754
loss: 0.163164153695106563 acc:  0.9674269153225806
loss: 0.037946645170450212 acc:  0.970703125
loss: 0.017000148072838783 acc:  0.9719632056451614
loss: 0.0143291288986802155 acc:  0.9595724126344086
loss: 0.035745404660701754 

In [24]:
test_net = Net()

In [24]:
weight = 'E050.pth'
PATH = os.path.join(weights_root, weight)
test_net = Net()
test_net.to(device)
acc = 0
for i, batch in enumerate(test_loader):
    data = batch[0].to(device)
    label = batch[1].to(device)
    print(data.shape)
    print(data)
    test_net.load_state_dict(torch.load(PATH))
    output = test_net(data)
    acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
    

print(acc/(i+1))
    
    

torch.Size([1, 1, 3, 25])
tensor([[[[ 6.7339e+02,  7.7007e+02,  7.9092e+02,  6.8129e+02,  5.5608e+02,
            7.5180e+02,  6.2130e+02,  4.8817e+02,  7.9614e+02,  8.1705e+02,
            6.1864e+02,  4.4109e+02,  7.7786e+02,  5.5337e+02,  5.8736e+02,
           -1.0000e+00,  6.7617e+02, -1.0000e+00,  7.0479e+02,  5.9522e+02,
            5.9263e+02,  5.9519e+02,  3.7851e+02,  3.7843e+02,  4.3857e+02],
          [ 2.0361e+02,  2.9759e+02,  2.8456e+02,  3.9416e+02,  3.8892e+02,
            3.0286e+02,  4.3856e+02,  4.0731e+02,  5.5072e+02,  5.3775e+02,
            5.5341e+02,  7.4395e+02,  5.5598e+02,  4.8544e+02,  6.6041e+02,
           -1.0000e+00,  1.9318e+02, -1.0000e+00,  1.9854e+02,  7.0230e+02,
            6.9962e+02,  6.8912e+02,  7.7010e+02,  7.6481e+02,  7.7265e+02],
          [ 3.5833e-01,  2.3797e-01, -6.3656e-02, -3.7101e-01,  5.4040e-01,
            3.3484e-01,  3.3305e-01,  7.0645e-01, -2.8334e-01, -5.3913e-01,
           -5.7206e-01, -1.6452e-01, -1.4232e-01,  4.1981e-0

In [94]:
acc = 0
for i, batch in enumerate(test_loader):
    
    net.eval()
    data = batch[0].to(device)
    label = batch[1].to(device)
    
    output = net(data).squeeze()
    #print(output)
    #output = output.topk(1)[1].squeeze()
    print(torch.argmax(output, dim=1))
    print(label.int())
    print((torch.argmax(output, dim=1) == label.int()))
    acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
    
print(acc/(i+1))
        

       

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1], device='cuda:0',
       dtype=torch.int32)
tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True], d

In [89]:
a = np.array([1,2,3])
b = np.array([1,1,1])

print((a == b).mean())

0.3333333333333333
